In [15]:

pip install nltk


In [14]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
import pandas as pd

# Charger le jeu de données
data = pd.read_csv("movie_review[1].csv")

# Afficher les premières lignes pour vérification
print(data.head())


   fold_id cv_tag  html_id  sent_id  \
0        0  cv000    29590        0   
1        0  cv000    29590        1   
2        0  cv000    29590        2   
3        0  cv000    29590        3   
4        0  cv000    29590        4   

                                                text  tag  
0  films adapted from comic books have had plenty...  pos  
1  for starters , it was created by alan moore ( ...  pos  
2  to say moore and campbell thoroughly researche...  pos  
3  the book ( or " graphic novel , " if you will ...  pos  
4  in other words , don't dismiss this film becau...  pos  


In [16]:
# Définition des stop words et de la ponctuation
stop_words = set(stopwords.words('english'))
punctuation = set(string.punctuation)



In [34]:

def preprocess_text(text):
    # Convertir le texte en minuscules
    text = text.lower()



    # Tokenisation du texte en mots
    words = nltk.word_tokenize(text)

    # Suppression des mots vides et de la ponctuation
    filtered_words = [word for word in words if word not in stop_words and word not in punctuation]

    # Reconstitution du texte prétraité
    preprocessed_text = ' '.join(filtered_words)

    return preprocessed_text

# Appliquer le prétraitement aux données textuelles
data['Text_Preprocessed'] = data['text'].apply(preprocess_text)
# Afficher les premières lignes du dataframe pour vérifier
# Accéder à une cellule spécifique contenant le texte prétraité et l'imprimer
print(data.head())

   fold_id cv_tag  html_id  sent_id  \
0        0  cv000    29590        0   
1        0  cv000    29590        1   
2        0  cv000    29590        2   
3        0  cv000    29590        3   
4        0  cv000    29590        4   

                                                text  tag  \
0  films adapted from comic books have had plenty...  pos   
1  for starters , it was created by alan moore ( ...  pos   
2  to say moore and campbell thoroughly researche...  pos   
3  the book ( or " graphic novel , " if you will ...  pos   
4  in other words , don't dismiss this film becau...  pos   

                                   Text_Preprocessed  
0  films adapted comic books plenty success wheth...  
1  starters created alan moore eddie campbell bro...  
2  say moore campbell thoroughly researched subje...  
3  book `` graphic novel `` 500 pages long includ...  
4                      words n't dismiss film source  


In [36]:
pip install gensim


In [40]:
from gensim.models import Word2Vec
# Appliquer le prétraitement aux données textuelles et former la liste de listes de mots
sentences = data['text'].apply(preprocess_text).tolist()


In [41]:
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

In [42]:
# Sauvegarde du modèle entraîné
model.save("word2vec_model.bin")

In [43]:
def get_review_vector(review):
    # Tokenisation du texte en mots
    words = nltk.word_tokenize(review.lower())

    # Filtrage des mots vides et de la ponctuation
    filtered_words = [word for word in words if word not in stop_words and word not in punctuation]

    # Initialiser le vecteur moyen à zéro
    review_vector = np.zeros(model.vector_size)

    # Calcul de la somme des embeddings des mots présents dans le modèle
    word_count = 0
    for word in filtered_words:
        if word in model.wv:
            review_vector += model.wv[word]
            word_count += 1

    # Si au moins un mot dans la review est présent dans le modèle, diviser le vecteur moyen par le nombre de mots
    if word_count != 0:
        review_vector /= word_count

    return review_vector

In [47]:
import numpy as np
# Appliquer la fonction get_review_vector à chaque review dans le dataframe
data['Review_Vector'] = data['text'].apply(get_review_vector)

# Afficher les premières lignes du dataframe avec les vecteurs de reviews
print(data)

       fold_id cv_tag  html_id  sent_id  \
0            0  cv000    29590        0   
1            0  cv000    29590        1   
2            0  cv000    29590        2   
3            0  cv000    29590        3   
4            0  cv000    29590        4   
...        ...    ...      ...      ...   
64715        9  cv999    14636       20   
64716        9  cv999    14636       21   
64717        9  cv999    14636       22   
64718        9  cv999    14636       23   
64719        9  cv999    14636       24   

                                                    text  tag  \
0      films adapted from comic books have had plenty...  pos   
1      for starters , it was created by alan moore ( ...  pos   
2      to say moore and campbell thoroughly researche...  pos   
3      the book ( or " graphic novel , " if you will ...  pos   
4      in other words , don't dismiss this film becau...  pos   
...                                                  ...  ...   
64715  that lack of inspirat

In [48]:
from sklearn.model_selection import train_test_split

# Diviser le dataset en ensembles d'entraînement et de test
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Afficher la taille des ensembles d'entraînement et de test
print("Taille de l'ensemble d'entraînement :", len(train_data))
print("Taille de l'ensemble de test :", len(test_data))


Taille de l'ensemble d'entraînement : 51776
Taille de l'ensemble de test : 12944


In [51]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Extraire les caractéristiques d'entrée (input features) et les étiquettes (labels)
X_train = np.array(train_data['Review_Vector'].to_list())
y_train = train_data['tag']

X_test = np.array(test_data['Review_Vector'].to_list())
y_test = test_data['tag']

# Entraîner le modèle de régression logistique
model = LogisticRegression(max_iter=1000)  # Vous pouvez ajuster les hyperparamètres selon vos besoins
model.fit(X_train, y_train)

# Faire des prédictions sur l'ensemble de test
y_pred = model.predict(X_test)

# Calculer la précision du modèle
accuracy = accuracy_score(y_test, y_pred)
print("Précision du modèle de régression logistique :", accuracy)


Précision du modèle de régression logistique : 0.507725587144623


In [52]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calculer les prédictions sur l'ensemble de test
y_pred = model.predict(X_test)

# Calculer les différentes métriques d'évaluation
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Afficher les métriques d'évaluation
print("Accuracy :", accuracy)
print("Precision :", precision)
print("Recall :", recall)
print("F1-score :", f1)


Accuracy : 0.507725587144623
Precision : 0.5028646402099018
Recall : 0.507725587144623
F1-score : 0.3561593879729367
